<a href="https://colab.research.google.com/github/dimash-web/child-health-data-feature/blob/main/Child_health_data_feature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Feature: Child Health Data

This feature allows a user to pick a metric related to children's welfare, like mortality rate, nutrition, etc. Then, we use the APIs to displays the related welfare stats of all countries on an interactive world map and a tableview.

## We will use two APIs:

The World Health Organization API to access specific metrics of children's welfare around the world.
An API to convert the country codes provided by the WHO into country names.

The API setup does not involve adding API Keys into the SECRETS tab.

## WORLD HEALTH ORGANIZATION API Usage:

Display all indicators related to children's welfare. Allow user to choose 1.
Input the indicator code as a parameter to retrieve the stats
Sort stats from high to low

In [5]:
#handle imports
import requests
import json
import pandas as pd

In [6]:
#search for all indicators pertaining to "Children" through the filter endpoint to select 1 indicator of interest.
url = f"https://ghoapi.azureedge.net/api/Indicator?$filter=contains(IndicatorName,'Children')"
response = requests.get(url)
if response.status_code == 200:
 data = response.json()
 print("Status: Successful!")
 print(data)
 print(json.dumps(data, indent=4))
else:
  print("Error")

Status: Successful!
{'@odata.context': 'https://ghoapi.azureedge.net/api/$metadata#Indicator', 'value': [{'IndicatorCode': 'AIR_10', 'IndicatorName': "Ambient air pollution  attributable DALYs per 100'000 children under 5 years", 'Language': 'EN'}, {'IndicatorCode': 'AIR_12', 'IndicatorName': 'Household air pollution attributable deaths in children under 5 years', 'Language': 'EN'}, {'IndicatorCode': 'AIR_16', 'IndicatorName': 'Household air pollution attributable DALYs in children under 5 years', 'Language': 'EN'}, {'IndicatorCode': 'AIR_18', 'IndicatorName': "Household air pollution  attributable DALYs per 100'000 children under 5 years", 'Language': 'EN'}, {'IndicatorCode': 'AIR_14', 'IndicatorName': "Household air pollution  attributable deaths per 100'000 children under 5 years", 'Language': 'EN'}, {'IndicatorCode': 'AIR_36', 'IndicatorName': 'Joint effects of air pollution attributable deaths in children under 5 years', 'Language': 'EN'}, {'IndicatorCode': 'AIR_4', 'IndicatorName

In [7]:
def display_indicators(data): #allow user to choose their metric of interest through a catalog
  indicators = data['value']
  for idx, indicator in enumerate(indicators):
    print(f"{idx + 1}: {indicator['IndicatorName']}")
  choice = int(input("\nEnter the number of the indicator you want to select: ")) - 1
  if 0 <= choice < len(indicators):
    selected_indicator = indicators[choice]
    print(f"\nYou selected: {selected_indicator['IndicatorName']}")
    print(f"Indicator Code: {selected_indicator['IndicatorCode']}")
    return selected_indicator['IndicatorCode'], selected_indicator['IndicatorName']
  else:
    print("You must choose a valid number!")

indicator_code, indicator_name = display_indicators(data)

1: Ambient air pollution  attributable DALYs per 100'000 children under 5 years
2: Household air pollution attributable deaths in children under 5 years
3: Household air pollution attributable DALYs in children under 5 years
4: Household air pollution  attributable DALYs per 100'000 children under 5 years
5: Household air pollution  attributable deaths per 100'000 children under 5 years
6: Joint effects of air pollution attributable deaths in children under 5 years
7: Ambient air pollution attributable deaths in children under 5 years
8: Reported number of children receiving antiretroviral therapy
9: Reported number of children receiving antiretroviral therapy, month and year of report
10: Estimated number of children needing antiretroviral therapy based on WHO methods
11: Estimated antiretroviral therapy coverage among children
12: Household and ambient air pollution attributable DALYs in children under 5 years ('000)
13: Ambient air pollution  attributable deaths per 100'000 children

In [8]:
indicator_code

'AIR_75'

In [9]:
# chose "Morality rate among children ages 10 to 14 years (per 1000 children aged 10)"
def get_data(indicator_code):
 url = f'https://ghoapi.azureedge.net/api/{indicator_code}'
 response = requests.get(url)
 if response.status_code == 200:
  print("Status: Successful!")
  data = response.json()
  print(json.dumps(data,indent=4))
  return data
 else:
  print("Error")

data = get_data(indicator_code)

Status: Successful!
{
    "@odata.context": "https://ghoapi.azureedge.net/api/$metadata#AIR_75",
    "value": [
        {
            "Id": 857577,
            "IndicatorCode": "AIR_75",
            "SpatialDimType": "COUNTRY",
            "SpatialDim": "GTM",
            "TimeDimType": "YEAR",
            "ParentLocationCode": "AMR",
            "ParentLocation": "Americas",
            "Dim1Type": "SEX",
            "Dim1": "SEX_FMLE",
            "TimeDim": 2019,
            "Dim2Type": "ENVCAUSE",
            "Dim2": "ENVCAUSE_ENVCAUSE039",
            "Dim3Type": null,
            "Dim3": null,
            "DataSourceDimType": null,
            "DataSourceDim": null,
            "Value": "22 229 [15 017-28 861]",
            "NumericValue": 22229.11646,
            "Low": 15017.28992,
            "High": 28860.76978,
            "Comments": null,
            "Date": "2022-08-26T11:40:15+02:00",
            "TimeDimensionValue": "2019",
            "TimeDimensionBegin": "2019-01-01

In [10]:
def process_data(data): #process_data is used to
    spatial_values = {}  #create an empty dictionary to later store spatial_dim (key) - numeric_value (value)
    for entry in data['value']: #loop through each country
     #if 'SpatialDim' in entry and 'NumericValue' in entry: #just in case these keys don't appear, but they do appear
     spatial_dim = entry['SpatialDim'] #retrieve the country code (its key name is SpatialDim)
     numeric_value = entry['NumericValue'] #retrieve the child mortality rate
     if spatial_dim not in spatial_values:
       spatial_values[spatial_dim] = numeric_value #append each country code and mortality rate to the dictionary

#sort by mortality rate in descending manner using the dictionary above
    sorted_spatial_values = sorted(spatial_values.items(), key=lambda x: x[1], reverse=True)
    return sorted_spatial_values

#call on the function
sorted_spatial_data = process_data(data)

#testing
for spatial_dim, value in sorted_spatial_data:
  print(f"Country Code: {spatial_dim}, {indicator_name}: {value:.2f}")

Country Code: NGA, Household air pollution attributable DALYs in children under 15 years: 3273883.34
Country Code: COD, Household air pollution attributable DALYs in children under 15 years: 1801729.86
Country Code: IND, Household air pollution attributable DALYs in children under 15 years: 1198425.27
Country Code: ETH, Household air pollution attributable DALYs in children under 15 years: 937216.35
Country Code: PAK, Household air pollution attributable DALYs in children under 15 years: 915426.85
Country Code: TZA, Household air pollution attributable DALYs in children under 15 years: 572577.81
Country Code: CMR, Household air pollution attributable DALYs in children under 15 years: 370822.50
Country Code: TCD, Household air pollution attributable DALYs in children under 15 years: 306196.87
Country Code: AGO, Household air pollution attributable DALYs in children under 15 years: 248320.27
Country Code: CIV, Household air pollution attributable DALYs in children under 15 years: 235382.

## REST COUNTRIES API USAGE:

Convert Country Code into Country Name (since WHO does not provide Country Name)
Then Merge with the output from WHO API.

In [11]:
country_codes = [] #create an empty list to store country codes
for spatial_dim, _ in sorted_spatial_data: #separate country code from mortality rate in spatial_dim
    country_codes.append(spatial_dim)

print("Country Codes:", country_codes)

Country Codes: ['NGA', 'COD', 'IND', 'ETH', 'PAK', 'TZA', 'CMR', 'TCD', 'AGO', 'CIV', 'MLI', 'SOM', 'MOZ', 'BEN', 'NER', 'BGD', 'BFA', 'UGA', 'CHN', 'MMR', 'GHA', 'IDN', 'KEN', 'HTI', 'MDG', 'AFG', 'GIN', 'ZMB', 'SDN', 'YEM', 'SLE', 'SSD', 'PHL', 'ZWE', 'VNM', 'BDI', 'SEN', 'LBR', 'MWI', 'CAF', 'RWA', 'ZAF', 'TGO', 'MEX', 'NPL', 'ERI', 'MRT', 'COG', 'GTM', 'GNQ', 'GNB', 'KHM', 'LAO', 'GMB', 'PNG', 'TJK', 'BRA', 'PRK', 'PER', 'RUS', 'UZB', 'BOL', 'TLS', 'LSO', 'DJI', 'PRY', 'IRN', 'IRQ', 'COM', 'NAM', 'NIC', 'MAR', 'HND', 'THA', 'DOM', 'KGZ', 'SWZ', 'EGY', 'BWA', 'ECU', 'COL', 'SYR', 'LKA', 'MNG', 'VEN', 'ROU', 'FJI', 'AZE', 'KAZ', 'PAN', 'GAB', 'STP', 'TUR', 'DZA', 'SLB', 'VUT', 'SLV', 'BTN', 'MYS', 'UKR', 'KIR', 'GUY', 'MDA', 'FSM', 'ARG', 'JAM', 'GEO', 'WSM', 'CUB', 'TKM', 'ARM', 'SRB', 'BIH', 'CPV', 'ALB', 'JOR', 'MKD', 'CRI', 'TUN', 'TON', 'SUR', 'BLZ', 'MUS', 'LCA', 'VCT', 'MNE', 'BLR', 'MDV', 'GRD', 'HRV', 'SVN', 'IRL', 'SWE', 'FRA', 'NOR', 'TTO', 'BHR', 'MLT', 'DNK', 'HUN', 'KOR

In [12]:
def get_countries_name(sorted_spatial_data): #decode country code into country name
    #use the all endpoint to fetch all attributes of a country (including name, code, etc.)
    url = 'https://restcountries.com/v3.1/all'
    response = requests.get(url)
    data = response.json()

    #create a mapping from country codes to country names
    country_code_map = {}
    for country in data:
     country_code_map[country['cca3']] = country['name']['common']
    return country_code_map


def merge_data(sorted_spatial_data): #merge results from 2 apis to get final output
  rows = []
  country_code_map = get_countries_name(sorted_spatial_data)
  for spatial_dim, value in sorted_spatial_data:
    country_name = country_code_map.get(spatial_dim, "Unknown")
    rows.append({'Country Name': country_name, 'Country Code': spatial_dim, indicator_name: value})
    print(f"Country Code: {spatial_dim}, Country Name: {country_name}, {indicator_name}: {value:.2f} ")
  df = pd.DataFrame(rows)
  return df

sorted_spatial_data = process_data(data)
df = merge_data(sorted_spatial_data)

Country Code: NGA, Country Name: Nigeria, Household air pollution attributable DALYs in children under 15 years: 3273883.34 
Country Code: COD, Country Name: DR Congo, Household air pollution attributable DALYs in children under 15 years: 1801729.86 
Country Code: IND, Country Name: India, Household air pollution attributable DALYs in children under 15 years: 1198425.27 
Country Code: ETH, Country Name: Ethiopia, Household air pollution attributable DALYs in children under 15 years: 937216.35 
Country Code: PAK, Country Name: Pakistan, Household air pollution attributable DALYs in children under 15 years: 915426.85 
Country Code: TZA, Country Name: Tanzania, Household air pollution attributable DALYs in children under 15 years: 572577.81 
Country Code: CMR, Country Name: Cameroon, Household air pollution attributable DALYs in children under 15 years: 370822.50 
Country Code: TCD, Country Name: Chad, Household air pollution attributable DALYs in children under 15 years: 306196.87 
Count

In [13]:
#create a clean tableview of countryname, countrycode, and their respective child moratlity rate
print(df)

    Country Name Country Code  \
0        Nigeria          NGA   
1       DR Congo          COD   
2          India          IND   
3       Ethiopia          ETH   
4       Pakistan          PAK   
..           ...          ...   
175        Spain          ESP   
176       Canada          CAN   
177       Kuwait          KWT   
178  Netherlands          NLD   
179        Italy          ITA   

     Household air pollution attributable DALYs in children under 15 years  
0                                         3.273883e+06                      
1                                         1.801730e+06                      
2                                         1.198425e+06                      
3                                         9.372163e+05                      
4                                         9.154268e+05                      
..                                                 ...                      
175                                       0.000000e+00          

In [14]:
#final output
#credit to ChatGPT for the beautiful and interactive visualization
import plotly.express as px
import numpy as np
import pandas as pd

country_codes = df['Country Code']
values = df[indicator_name]

# Create the choropleth map
fig = px.choropleth(
    df,
    locations='Country Code',
    locationmode='ISO-3',  # Use 'ISO-3' for ISO 3166-1 alpha-3 country codes
    color= indicator_name,
    color_continuous_scale='Viridis',
    title= indicator_name
)

# Update layout to enlarge the graph
fig.update_layout(
    width=1400,  # Set width (e.g., 1200 pixels)
    height=800   # Set height (e.g., 800 pixels)
)

# Show the figure
fig.show()

#display df
pd.set_option('display.max_rows', None)  # or a specific number like 1000
pd.set_option('display.max_columns', None)  # or a specific number like 50
df




,Country Name,Country Code,Household air pollution attributable DALYs in children under 15 years
0,Nigeria,NGA,3.273883e+06
1,DR Congo,COD,1.801730e+06
2,India,IND,1.198425e+06
3,Ethiopia,ETH,9.372163e+05
4,Pakistan,PAK,9.154268e+05
5,Tanzania,TZA,5.725778e+05
6,Cameroon,CMR,3.708225e+05
7,Chad,TCD,3.061969e+05
8,Angola,AGO,2.483203e+05
9,Ivory Coast,CIV,2.353829e+05
